In [4]:
! pip install import-ipynb

/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point


In [5]:
from model import BodyposeModel
import cv2
import numpy as np
import math
import time
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt
import matplotlib
import torch
from torchvision import transforms
import util
from torchsummary import summary
from body import Body
import import_ipynb
from train_model import BodyposeBackbone, MainModel
from coco_dataset import COCODataset, transform_image
from copy import deepcopy
import random

/tmp/ipykernel_36381/534042546.py:6: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


importing Jupyter notebook from train_model.ipynb
importing Jupyter notebook from coco_dataset.ipynb


In [6]:
NO_OF_OUTPUTS = 21

In [7]:
colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0], \
            [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], \
            [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85], [255, 255, 255]]

In [8]:
new_order = [0, 15, 14, 17, 16, 5, 2, 6, 3, 7, 4, 11, 8, 12, 9, 13, 10]

In [9]:
backbone_model_path = '../model/body_pose_model.pth'

In [10]:
backbone_model = BodyposeBackbone()

In [11]:
backbone_params = torch.load(backbone_model_path)

In [12]:
backbone_dict = util.transfer(backbone_model, backbone_params)

In [13]:
backbone_model.load_state_dict(backbone_dict)

<All keys matched successfully>

In [14]:
main_model = MainModel(NO_OF_OUTPUTS)

In [15]:
train_data = COCODataset('/media/jakub/One Touch/coco_pose/coco2017labels-pose/coco-pose', transforms=transform_image(split='train'))

loading annotations into memory...


Done (t=1.00s)
creating index...
index created!


In [16]:
skeleton = train_data.coco.cats[1]['skeleton']
skeleton = [[connection[0]-1, connection[1]-1] for connection in skeleton]

In [70]:
index = random.randint(0, 100)
anns = []
while anns == []:
    for id in train_data.coco.getAnnIds():
        if train_data.coco.anns[id]['image_id'] == train_data.coco.getImgIds()[index]:
            ann = train_data.coco.anns[id]
            if 1 in ann['keypoints'] or 2 in ann['keypoints']:
                anns.append(ann)
    if anns == []:
        index = random.randint(0, 100)



In [71]:
anns

[{'segmentation': [[362.86,
    41.22,
    338,
    57.78,
    334.69,
    60.27,
    331.38,
    70.21,
    333.86,
    89.26,
    334.69,
    95.89,
    330.55,
    95.89,
    325.58,
    92.58,
    323.92,
    100.03,
    325.58,
    109.15,
    321.44,
    116.6,
    317.29,
    121.57,
    311.49,
    134,
    307.35,
    143.11,
    306.52,
    154.71,
    300.72,
    166.31,
    288.3,
    182.05,
    282.5,
    182.88,
    274.21,
    190.33,
    267.59,
    186.19,
    264.27,
    180.39,
    259.3,
    182.05,
    259.3,
    194.48,
    256.82,
    211.87,
    260.96,
    212.7,
    273.39,
    209.39,
    284.98,
    209.39,
    307.35,
    200.28,
    313.98,
    195.31,
    318.95,
    188.68,
    329.72,
    166.31,
    334.69,
    158.85,
    343.8,
    150.57,
    347.12,
    163.82,
    340.49,
    201.1,
    334.69,
    219.33,
    333.86,
    221.82,
    334.69,
    242.53,
    339.66,
    250.81,
    342.15,
    254.95,
    343.8,
    273.18,
    348.77,
    285.61,

In [72]:
bboxes = [ann['bbox'] for ann in anns]

In [73]:
bboxes = [[int(element) for element in box] for box in bboxes]

In [74]:
keypoints = [ann['keypoints'] for ann in anns]

In [75]:
new_keypoints = []
for keypoint in keypoints:
    new_keypoints.append([[keypoint[i], keypoint[i+1], keypoint[i+2]] for i in range(0, len(keypoint), 3)])
keypoints = new_keypoints

In [76]:
image = train_data._load_image(train_data.coco.getImgIds()[index])
for bbox in bboxes:
    cv2.rectangle(image, (bbox[0], bbox[1]), (bbox[0]+bbox[2], bbox[1]+bbox[3]), (0, 0, 255))
for i, connection in enumerate(skeleton):
    for keypoint in keypoints:
        point1 = keypoint[connection[0]]
        point2 = keypoint[connection[1]]
        if point1[2] != 0 and point2[2] != 0:
            cv2.line(image, point1[:-1], point2[:-1], color=colors[i], thickness=3)
cv2.imshow('Example image', image)
cv2.waitKey(5_000)
cv2.destroyAllWindows()

In [77]:
def train(backbone_model, main_model, loss, optimizer, inputs, outputs):
    optimizer.zero_grad()
    main_model.train()

    out1 = backbone_model(inputs)
    logits = main_model(out1)
    output = loss.forward(logits, outputs)
    output.backward()
    optimizer.step()

    return output.item()

In [78]:
loss = torch.nn.MSELoss()
params = [p for p in main_model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params)

In [79]:
epochs = 10
batch_size = 4
num_of_batches = Xtrain_torch.size[0] // batch_size

for epoch in range(epochs):
    for batch in range(num_of_batches):
        inputs = Xtrain_torch[batch*batch_size:(batch+1)*batch_size]
        outputs = Ytrain_torch[batch*batch_size:(batch+1)*batch_size]
        output = train(backbone_model, main_model, loss, optimizer, inputs, outputs)

NameError: name 'Xtrain_torch' is not defined